# Zone based analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:,.0f}'.format

## read data
* od matrices: Prt, PuT

In [2]:
od_PuT = pd.read_csv('data//MTX_203.csv')
del od_PuT['Unnamed: 0']
od_PuT.columns = [int(c) for c in od_PuT.columns]
od_PrT = pd.read_csv('data//MTX_204.csv')
del od_PrT['Unnamed: 0']
od_PrT.columns = [int(c) for c in od_PrT.columns]

* trips

In [23]:
trips = pd.read_csv('data//From_Via_To.csv')
del trips['Unnamed: 0']
trips.head()

,Z_Rejon,POI,Do_Rejon,Czas_PuT,Czas_PrT
0,1,1,1,"2,284",520
1,1,1,2,"2,222",655
2,1,1,3,"2,222",637
3,1,1,4,"2,505",667
4,1,1,5,"3,014",785


## prepare

In [5]:
trips.Czas_PrT = trips.Czas_PrT/60 # przeliczam na minuty
trips.Czas_PuT = trips.Czas_PuT/60 *0.5 # tu zakladam dwa razy krotsze, dla obliczen, w Warszawie tak nie bedzie
BUDGET_PRT = trips['Czas_PrT'].quantile(0.5) # przyjmuje ze liczy sie tylko dolna polowa czasow
BUDGET_PUT = BUDGET_PRT #trips['Czas_PuT'].quantile(0.5) # a tu juz zakladam ze budzety sa rowne
trips['time_left_PrT']=BUDGET_PRT - trips.Czas_PrT # ile czasu zostaje ponizej budzetu w POI
trips['time_left_PuT']=BUDGET_PUT - trips.Czas_PuT
pd.options.display.float_format = '{:,.1f}'.format
trips.head()

,Z_Rejon,POI,Do_Rejon,Czas_PuT,Czas_PrT,time_left_PrT,time_left_PuT
0,1.0,1.0,1.0,19.0,8.7,3.2,-7.1
1,1.0,1.0,2.0,18.5,10.9,1.0,-6.6
2,1.0,1.0,3.0,18.5,10.6,1.3,-6.6
3,1.0,1.0,4.0,20.9,11.1,0.8,-9.0
4,1.0,1.0,5.0,25.1,13.1,-1.2,-13.2


Zone-based analysis:
- number of POI available
- total minutes
- mean/median minutes
- number of people

loop trips - z rejonu, do POI, z powrotem do tego samego rejonu.

In [6]:
loop_trips = trips[trips.Z_Rejon == trips.Do_Rejon]

In [7]:
print("Liczba POI dostepnych z rejonu w budzecie (podroz tam i z powrotem)")
nPOIS_z_Rejonu = loop_trips[loop_trips.Czas_PrT<BUDGET_PRT].groupby(by=['Z_Rejon'])['POI'].nunique().to_frame(name="PrT")
nPOIS_z_Rejonu['PuT'] = loop_trips[loop_trips.Czas_PuT<BUDGET_PUT].groupby(by=['Z_Rejon'])['POI'].nunique()
nPOIS_z_Rejonu

Liczba POI dostepnych z rejonu w budzecie (podroz tam i z powrotem)


,PrT,PuT
Z_Rejon,,
1.0,21,1.0
2.0,24,7.0
3.0,25,9.0
4.0,25,4.0
5.0,17,1.0
6.0,20,6.0
7.0,8,2.0
8.0,13,1.0
9.0,8,nan


* Zapis wynikow do .csv (mozesz tak zrobic dla kazdego obiektu data frame, a wiec wszystkich wynikow z tego jupytera)

In [24]:
nPOIS_z_Rejonu.to_csv('data/nPOIS_z_Rejonu.csv')

In [10]:
print("total minutes; mean, median minutes PrT - czas i time_left")
loop_trips[loop_trips.Czas_PrT<BUDGET_PRT].groupby(
    by=['Z_Rejon'])['Czas_PrT', 'time_left_PrT'].agg(['sum','mean','median'])

total minutes; mean, median minutes PrT - czas i time_left


Czas_PrT             time_left_PrT            
             sum mean median           sum mean median
Z_Rejon                                               
1.0        166.6  7.9    7.9          83.2  4.0    4.0
2.0        173.7  7.2    7.3         111.7  4.7    4.6
3.0        192.8  7.7    7.8         104.5  4.2    4.1
4.0        215.7  8.6    9.1          81.7  3.3    2.8
5.0        131.7  7.7    8.3          70.5  4.1    3.6
6.0        162.0  8.1    8.4          75.9  3.8    3.5
7.0         82.0 10.2   10.6          13.1  1.6    1.3
8.0        123.2  9.5    9.7          31.4  2.4    2.2
9.0         89.6 11.2   11.4           5.6  0.7    0.5
10.0       119.1  9.2    9.0          35.5  2.7    2.9
11.0        22.2 11.1   11.1           1.6  0.8    0.8
12.0        85.2 10.6   11.3          10.0  1.2    0.6
13.0        95.4 10.6   11.3          11.6  1.3    0.6
14.0        81.9  9.1    9.8          25.1  2.8    2.1
15.0       157.6  8.8    9.0          56.5  3.1    2.9
16.0        79.7  8.9    9.1          27.4  3.0    2.8
17.0       140.6  9.4   10.4          37.8  2.5    1.5
18.0       172.4  8.6    8.5          65.5  3.3    3.4
19.0       111.7  9.3    9.3          31.0  2.6    2.6
21.0       161.9  9.5    9.9          40.3  2.4    2.0
22.0       194.3  8.1    8.4          91.1  3.8    3.5
23.0       208.2  7.7    8.0         112.9  4.2    3.9
24.0       198.2  7.6    7.7         111.1  4.3    4.2

In [7]:
print("Number of origin/destination WH trips daily")
n_trips = od_PrT.sum(axis=1).to_frame('PrT_dest')
n_trips['PrT_orig'] = od_PrT.sum(axis=0)
n_trips['PuT_dest'] = od_PuT.sum(axis=1)
n_trips['PuT_orig'] = od_PuT.sum(axis=0)
n_trips

Number of origin/destination WH trips daily


,PrT_dest,PrT_orig,PuT_dest,PuT_orig
0,37,nan,26,nan
1,497,73,344,51
2,372,"1,353",259,942
3,84,"1,192",57,825
4,782,44,539,30
5,218,425,153,298
6,54,"1,316",36,918
7,368,220,264,150
8,46,414,32,296
9,292,189,203,129
